In [1]:
! wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
! tar -xvzf ml-20m-compact.tar.gz
! wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-features.tar.gz
! tar -xvzf ml-20m-features.tar.gz
! pip install caserecommender

--2022-05-06 18:53:13--  https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mmanzato/MBABigData/master/ml-20m-compact.tar.gz [following]
--2022-05-06 18:53:13--  https://raw.githubusercontent.com/mmanzato/MBABigData/master/ml-20m-compact.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65019041 (62M) [application/octet-stream]
Saving to: ‘ml-20m-compact.tar.gz’

ml-20m-compact.tar. 100%[===================>]  62.01M   356MB/s    in 0.2s    

2022-05-06 18:53:18 (356 MB/s) - ‘ml-20m-compact.tar.gz’ saved [65019041/65019041

# Processando features visuais

In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
with open('./features/visual_histogram_50.arq', 'rb') as arq_visualHistograms:
    visualHistograms = pickle.load(arq_visualHistograms)
len(visualHistograms)

433

In [4]:
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine

#Valid metrics are ['euclidean', 'l2', 'l1', 'manhattan', 'cityblock', 'braycurtis', 'canberra', 'chebyshev', 'correlation', 'cosine', 'dice', 'hamming', 'jaccard', 'kulsinski', 'mahalanobis', 'matching', 'minkowski', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule', 'wminkowski', 'nan_euclidean', 'haversine'], or 'precomputed', or a callable
sim_v_matrix = pairwise_distances(visualHistograms, metric="cosine")
sim_v_matrix.shape

(433, 433)

# Ler e preparar os dados

In [5]:
movies = pd.read_csv('./dataset/movies_sample.csv')
movies.head()

,movieId,title,genres
0,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Crime|Drama
1,31,Dangerous Minds (1995),Drama
2,37,Across the Sea of Time (1995),Documentary|IMAX
3,161,Crimson Tide (1995),Drama|Thriller|War
4,193,Showgirls (1995),Drama


In [6]:
with open('sim_v_matrix_raw_50.dat', 'w') as arq_sim_v_matrix:
  for i in range(len(visualHistograms)):
    for j in range(len(visualHistograms)):
      if i < j:
        arq_sim_v_matrix.write(str(movies.iloc[i].movieId) + '\t' + str(movies.iloc[j].movieId) + '\t' + str(sim_v_matrix[i][j]) + '\n')


In [7]:
from sklearn.model_selection import train_test_split

ratings = pd.read_csv('./dataset/ratings_sample.csv')
df = ratings[['userId', 'movieId', 'rating']]
df = df.merge(movies[['movieId', 'title']])
map_users = {user: idx for idx, user in enumerate(df.userId.unique())}
map_items = {item: idx for idx, item in enumerate(df.movieId.unique())}
df['userId'] = df['userId'].map(map_users)
df['movieId'] = df['movieId'].map(map_items)
train, test = train_test_split(df, test_size=.2, random_state=2)
train.to_csv('train.dat', index=False, header=False, sep='\t')
test.to_csv('test.dat', index=False, header=False, sep='\t')

map_title = {}
for _, row in df.iterrows():
    map_title[row.movieId] = row.title

In [8]:
sim = pd.read_csv('sim_v_matrix_raw_50.dat', sep='\t', header=0, names=['movieId1', 'movieId2', 'similarity'])
sim['movieId1'] = sim['movieId1'].map(map_items)
sim['movieId2'] = sim['movieId2'].map(map_items)
sim.dropna(inplace=True)
sim['movieId1'] = sim.movieId1.astype(int)
sim['movieId2'] = sim.movieId2.astype(int)
sim[['movieId1', 'movieId2', 'similarity']].to_csv('sim_v_matrix_50.dat', index=False, sep='\t', header=False)

### Case Recommender


In [9]:
from caserec.recommenders.rating_prediction.item_attribute_knn import ItemAttributeKNN

In [10]:
ItemAttributeKNN('train.dat', 'test.dat', similarity_file='sim_v_matrix_50.dat', as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 403 items (152496 interactions) | sparsity:: 96.59%
test data:: 10503 users and 340 items (38125 interactions) | sparsity:: 98.93%

training_time:: 6.053403 sec
prediction_time:: 0.721492 sec
Eval:: MAE: 0.726465 RMSE: 0.956365 


# Processando features aurais

In [11]:
with open('./features/aural_histogram_50_sn.arq', 'rb') as arq_auralHistograms:
    auralHistograms = pickle.load(arq_auralHistograms)
len(auralHistograms)

433

In [12]:
sim_a_matrix = pairwise_distances(auralHistograms, metric="cosine")
sim_a_matrix.shape

(433, 433)

In [13]:
with open('sim_a_matrix_raw_50.dat', 'w') as arq_sim_a_matrix:
  for i in range(len(auralHistograms)):
    for j in range(len(auralHistograms)):
      if i < j:
        arq_sim_a_matrix.write(str(movies.iloc[i].movieId) + '\t' + str(movies.iloc[j].movieId) + '\t' + str(sim_a_matrix[i][j]) + '\n')


In [14]:
sim = pd.read_csv('sim_a_matrix_raw_50.dat', sep='\t', header=0, names=['movieId1', 'movieId2', 'similarity'])
sim['movieId1'] = sim['movieId1'].map(map_items)
sim['movieId2'] = sim['movieId2'].map(map_items)
sim.dropna(inplace=True)
sim['movieId1'] = sim.movieId1.astype(int)
sim['movieId2'] = sim.movieId2.astype(int)
sim[['movieId1', 'movieId2', 'similarity']].to_csv('sim_a_matrix_50.dat', index=False, sep='\t', header=False)

### Case Recommender

In [15]:
ItemAttributeKNN('train.dat', 'test.dat', similarity_file='sim_a_matrix_50.dat', as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 403 items (152496 interactions) | sparsity:: 96.59%
test data:: 10503 users and 340 items (38125 interactions) | sparsity:: 98.93%

training_time:: 6.017461 sec
prediction_time:: 0.352205 sec
Eval:: MAE: 0.711012 RMSE: 0.937911 


# Processando features multimodais

In [16]:
with open('./features/mm_sum_histogram_50_sn.arq', 'rb') as arq_mm_sum_histograms:
    mmSumHistograms = pickle.load(arq_mm_sum_histograms)
len(mmSumHistograms)

433

In [17]:
sim_mm_sum_matrix = pairwise_distances(mmSumHistograms, metric="cosine")
sim_mm_sum_matrix.shape

(433, 433)

In [18]:
with open('sim_mm_sum_matrix_raw_50.dat', 'w') as arq_sim_mm_sum_matrix:
  for i in range(len(mmSumHistograms)):
    for j in range(len(mmSumHistograms)):
      if i < j:
        arq_sim_mm_sum_matrix.write(str(movies.iloc[i].movieId) + '\t' + str(movies.iloc[j].movieId) + '\t' + str(sim_mm_sum_matrix[i][j]) + '\n')

In [19]:
sim = pd.read_csv('sim_mm_sum_matrix_raw_50.dat', sep='\t', header=0, names=['movieId1', 'movieId2', 'similarity'])
sim['movieId1'] = sim['movieId1'].map(map_items)
sim['movieId2'] = sim['movieId2'].map(map_items)
sim.dropna(inplace=True)
sim['movieId1'] = sim.movieId1.astype(int)
sim['movieId2'] = sim.movieId2.astype(int)
sim[['movieId1', 'movieId2', 'similarity']].to_csv('sim_mm_sum_matrix_50.dat', index=False, sep='\t', header=False)

### Case Recommender

In [20]:
ItemAttributeKNN('train.dat', 'test.dat', similarity_file='sim_mm_sum_matrix_50.dat', as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 403 items (152496 interactions) | sparsity:: 96.59%
test data:: 10503 users and 340 items (38125 interactions) | sparsity:: 98.93%

training_time:: 6.069139 sec
prediction_time:: 0.477231 sec
Eval:: MAE: 0.713407 RMSE: 0.94181 


# Fusão de features visuais e aurais

### Combinando matrizes de similaridade

In [21]:
sim_aural = pd.read_csv('sim_a_matrix_50.dat', sep='\t', names=['movieId1', 'movieId2', 'similarity'])
sim_visual = pd.read_csv('sim_v_matrix_50.dat', sep='\t', names=['movieId1', 'movieId2', 'similarity'])

In [22]:
sim_fus = sim_aural.append(sim_visual).groupby(['movieId1', 'movieId2'],as_index=False).agg({'similarity':'max'})
sim_fus[['movieId1', 'movieId2', 'similarity']].to_csv('sim_f_matrix_50.dat', index=False, sep='\t', header=False)

In [23]:
ItemAttributeKNN('train.dat', 'test.dat', similarity_file='sim_f_matrix_50.dat', as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 403 items (152496 interactions) | sparsity:: 96.59%
test data:: 10503 users and 340 items (38125 interactions) | sparsity:: 98.93%

training_time:: 6.170216 sec
prediction_time:: 0.597986 sec
Eval:: MAE: 0.726413 RMSE: 0.956272 


### Combinando vetores de features

In [26]:
with open('./features/aural_histogram_50_sn.arq', 'rb') as arq_auralHistograms:
    auralHistograms = pickle.load(arq_auralHistograms)
with open('./features/visual_histogram_50_sn.arq', 'rb') as arq_visualHistograms:
    visualHistograms = pickle.load(arq_visualHistograms)

In [27]:
merge = np.concatenate((visualHistograms,auralHistograms), axis=1)
sim_ff_matrix = pairwise_distances(merge, metric="cosine")

with open('sim_ff_matrix_raw_50.dat', 'w') as arq_sim_ff_matrix:
  for i in range(len(merge)):
    for j in range(len(merge)):
      if i < j:
        arq_sim_ff_matrix.write(str(movies.iloc[i].movieId) + '\t' + str(movies.iloc[j].movieId) + '\t' + str(sim_ff_matrix[i][j]) + '\n')

sim = pd.read_csv('sim_ff_matrix_raw_50.dat', sep='\t', header=0, names=['movieId1', 'movieId2', 'similarity'])
sim['movieId1'] = sim['movieId1'].map(map_items)
sim['movieId2'] = sim['movieId2'].map(map_items)
sim.dropna(inplace=True)
sim['movieId1'] = sim.movieId1.astype(int)
sim['movieId2'] = sim.movieId2.astype(int)
sim[['movieId1', 'movieId2', 'similarity']].to_csv('sim_ff_matrix_50.dat', index=False, sep='\t', header=False)

In [28]:
ItemAttributeKNN('train.dat', 'test.dat', similarity_file='sim_ff_matrix_50.dat', as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 403 items (152496 interactions) | sparsity:: 96.59%
test data:: 10503 users and 340 items (38125 interactions) | sparsity:: 98.93%

training_time:: 6.063032 sec
prediction_time:: 0.475278 sec
Eval:: MAE: 0.711094 RMSE: 0.937899 
